In [9]:
import pandas as pd
import pickle as pk
from sklearn.utils import Bunch
import numpy as np

In [2]:
#Creamos el diccionario con la estructura indicada
filepath = '../PC-GITA/PCGITA_metadata.csv'
dic_audios_inf = dict()
with open(filepath) as fp:
    cnt = 0
    for line in fp:
        line = line.split(';')[:8]
        if not line[0].startswith('AVP'):
            line[0]=line[0][3:]
        dic_audios_inf[line[0]]=dict()
        dic_audios_inf[line[0]]['UPDRS'] = 0 if line[1]=='' else int(line[1])
        dic_audios_inf[line[0]]['UPDRS_SPEECH'] = 0 if line[2]=='' else int(line[2])
        dic_audios_inf[line[0]]['HY'] = 0 if line[3]=='' else int(line[3][0])
        dic_audios_inf[line[0]]['SEX'] = 0 if line[4]=='M' else 1
        dic_audios_inf[line[0]]['AGE'] = int(line[5])
fp.close()
metadata_df = pd.DataFrame(dic_audios_inf).transpose()
metadata_df.head(10)

,AGE,HY,SEX,UPDRS,UPDRS_SPEECH
AVPEPUDEA0001,64,2,0,28,1
AVPEPUDEA0002,72,1,1,19,0
AVPEPUDEA0003,75,3,1,52,2
AVPEPUDEA0005,65,2,0,32,1
AVPEPUDEA0006,66,2,1,28,1
AVPEPUDEA0007,55,2,1,30,1
AVPEPUDEA0008,60,2,1,29,1
AVPEPUDEA0009,57,3,1,41,1
AVPEPUDEA0010,51,3,1,38,2
AVPEPUDEA0011,55,3,1,43,2


**Extraeremos las características de VGGish embeddings para los read-text y para cada una de las vocales.**

> Audio -->**|Preprocesado|**--> MFCC y espectros para cada segundo -->**|VGGish|**-->Embeddings-->**|Media y desviacion|**--> final embeddings

##  Extracción VGGish embeddings read-text

**Todos los audios de Read Text satisfacen ser mayores de 0.95 segundos, por lo que la librería procesará todos sin descartar ninguno**.

In [3]:
from extractor_ccas_vggish import Extractor_Caracteristicas_Vggish

Using TensorFlow backend.


In [4]:
rutaCcas = 'CaracteristicasExtraidas/Regression/embbedings/'
extractor = Extractor_Caracteristicas_Vggish(rutaCcas,dic_audios_inf)

Directorio de características ya existente, no se crea nuevo.



In [5]:
vggish_embed_rt_ccas = extractor.extraccion_embeddings_directorio('PC-GITA/read-text/',['SEX','AGE'])

Comienzo extracción HC, quedan: 100 audios.

Comienzo extracción PD, quedan: 50 audios.


In [6]:
vggish_embed_rt_ccas.shape

(100, 259)

In [7]:
pd.DataFrame(vggish_embed_rt_ccas).head()

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258
0,0.278153,0.012632,0.107849,0.003633,0.002978,0.00377,0.019045,0.000866,0.004500,0.037274,...,0.434527,0.000000,0.118272,0.119562,0.158425,0.081016,0.000000,0.0,64.0,1.0
1,0.325714,0.034539,0.436125,0.034923,0.010663,0.00000,0.001808,0.000000,0.030295,0.013750,...,0.059658,0.000000,0.318470,0.276926,0.000000,0.013921,0.058811,1.0,72.0,1.0
2,0.264087,0.020774,0.381020,0.005155,0.008006,0.00000,0.012588,0.000000,0.001721,0.003728,...,0.260098,0.015644,0.292940,0.291935,0.022067,0.026897,0.034241,1.0,75.0,1.0
3,0.401916,0.011046,0.207072,0.016177,0.000000,0.00000,0.000000,0.001114,0.000000,0.029870,...,0.098720,0.000000,0.151806,0.220594,0.207069,0.004790,0.044313,0.0,65.0,1.0
4,0.850589,0.000000,0.234680,0.000000,0.024994,0.00000,0.000000,0.000000,0.015651,0.053887,...,0.146179,0.000000,0.140579,0.245352,0.010431,0.000000,0.000000,1.0,66.0,1.0


In [10]:
bunch = Bunch(data = vggish_embed_rt_ccas[:,:-1], 
                  target = vggish_embed_rt_ccas[:,-1], 
                  UPDRS = np.array(metadata_df['UPDRS']), 
                  UPDRS_SPEECH = np.array(metadata_df['UPDRS_SPEECH']), 
                  HY = np.array(metadata_df['HY']) )
    
pickle_out = open('../CaracteristicasExtraidas/Regression/embbedings/vggish_embed_rt_ccas.bunch',"wb")
pk.dump(bunch, pickle_out)
pickle_out.close()

##  Extracción VGGish embeddings VOCALES

**NO todos los audios de Read Text satisfacen ser mayores de 0.95 segundos, por lo que la librería descartará audios automáticamente**.

**Debemos procesar audio a audio para poder insertar correctamente las labels**.

In [11]:
import numpy as np
import vggish_input
import vggish_params
import vggish_postprocess
import vggish_keras
import os
from sklearn.utils import Bunch
    
class Ex_Ccas_Vggish:
    
    ''' 
    Clase encargada de la extracción de características de los audios con la herramienta VGGish 

    author: Adrián Arnaiz

    Attributes
    ----------
    rutaCcas: str
        ruta donde guardar los archivos de características
    dic_inf_audios: dict
        diccionario donde se guardan atributos extra para cada audio.
        Clave: nombre del audio. Valor: Diccionario con claves el nombre de atributo y valor el valor.
    '''

    def __init__(self, rutaCarac, dic_inf_audios=None):

        '''
        Paramteros
        ----------
        rutaCarac: str
            ruta donde guardar los archivos de características
        dic_inf_audios: dict
            diccionario donde se guardan atributos extra para cada audio.
            Clave: nombre del audio. Valor: Diccionario con claves el nombre de atributo y valor el valor.
        '''

        self.rutaCcas = '../'+rutaCarac #i.e.: CaracteristicasExtraidas/Vggish/embeddings||espectros/
        
        try: 
            os.mkdir(self.rutaCcas)
        except FileExistsError:
            print('Directorio de características ya existente, no se crea nuevo.')
        
        self.dic_inf_audios = dic_inf_audios
        
        #Definimos VGGish
        self.model = vggish_keras.get_vggish_keras()
        #Cargamos el checkpoint
        checkpoint_path = 'vggish_weights.ckpt'
        self.model.load_weights(checkpoint_path)
        
        
    def add_target(self, ccas, parkinson):
        ''' 
        Añade la colmuna target. PD: 1, HC: 0. 
        
        Parametros
        ----------
        ccas : numpy.array 2 dimensiones
            matriz de ccas de audios a la que añadir la última columna
        parkinson: bool
            booleano indicando si se quiere añadir columna parkinson o no

        Returns
        -------
            matriz con la clase añadida.
        '''
        return np.hstack((ccas,np.ones((ccas.shape[0],1)))) if parkinson else np.hstack((ccas,np.zeros((ccas.shape[0],1))))
    
    def extraccion_embeddings_directorio(self, aud, extra_atribs=None, embeddings = True):
        '''
        Devuelve en numpy las medidas de los todos audios de un directorio que saca el VGGish y se guardan en el archivo ccas.npy.
        Recorre para un tipo de audio primero los sanos y los etiqueta y posteriormente hace lo mismo con los PD.
        Finalmente los concatena.


        Parametros
        ----------
        aud: str
            ruta del directorio de audios a analizar respecto a src/. i.e: 'PC-GITA/read-text/'. Debe subcontener hc y pd.
        extra_atribs: list(string)
            atributos extra a añadir a las caracteriticas. i.e. edad o sexo. lista con los nombres 
            de los atributos a añadir del self.dic_inf_audios
        embeddings:boolean
            Si es true sacamos los embeddings, si false sacamos los espectros

        Returns
        -------
            matriz de instancias con sus ccas extraidas y atributos añadidos si fuese necesario.
            Serán embeddings o espectros en función de el parámetro boolean.
        '''
        rutaAudios = '../'+aud 

        
        ######
        ##HC##
        ######
        rutaAudiosTipo = rutaAudios+'hc/'
        audios = os.listdir(rutaAudiosTipo)
        print('Comienzo extracción HC, quedan:', len(audios)*2,'audios.')        
        
        ccas_hc=[]

        for audio in audios:
            embedd = self.extraccion_embeddings_audio(rutaAudiosTipo+audio, embeddings)
            
            #Si es nan tiene shape 2, ponemos shape igual a los demás para evitar fallos
            if(np.isnan(embedd).any()):
                embedd = np.empty((256,))
                embedd[:] = np.NaN
            
            #Cogemos los metadatos y añadimos edad y sexo como atributo
            metadata = self.dic_inf_audios[audio[:-6]]
            embedd = np.append(embedd, 
                               [metadata['SEX'],
                                metadata['AGE'],
                                0,
                                metadata['UPDRS'],
                                metadata['UPDRS_SPEECH'],
                                metadata['HY']
                               ])
            
            assert embedd.shape[0]==256+2+4
            
            #Añadimos las ccas a la matriz de ccas
            ccas_hc.append(embedd)
            
        ccas_hc=np.array(ccas_hc)

        #Borramos los audios de menos de un segundo, cuya salida de vggish es np.array([nan,nan],)
        ccas_hc = np.array([instancia for instancia in ccas_hc if not np.isnan(instancia).any()])

    
        ######
        ##PD##
        ######
        print('Comienzo extracción PD, quedan:', len(audios),'audios.')

        rutaAudiosTipo = rutaAudios+'pd/'
        audios = os.listdir(rutaAudiosTipo)
        
        ccas_pd=[]

        for audio in audios:
            embedd = self.extraccion_embeddings_audio(rutaAudiosTipo+audio, embeddings)
            
            #Si es nan tiene shape 2, ponemos shape igual a los demás para evitar fallos
            if(np.isnan(embedd).any()):
                embedd = np.empty((256,))
                embedd[:] = np.NaN
            
            #Cogemos los metadatos y añadimos edad y sexo como atributo
            metadata = self.dic_inf_audios[audio[:-6]]
            embedd = np.append(embedd, 
                               [metadata['SEX'],
                                metadata['AGE'],
                                1,
                                metadata['UPDRS'],
                                metadata['UPDRS_SPEECH'],
                                metadata['HY']
                               ])
            
            
            assert embedd.shape[0]==256+2+4
            
            #Añadimos las ccas a la matriz de ccas
            ccas_pd.append(embedd)
            
        ccas_pd=np.array(ccas_pd)

        #Limpiamos
        ccas_pd = np.array([instancia for instancia in ccas_pd if not np.isnan(instancia).any()])

        #Devolvemos todo el conjunto entero junto
        return np.concatenate((ccas_hc, ccas_pd))

  
    def extraccion_embeddings_audio(self, rutaAudio, embeddings):
        '''Extrae de un único audio, la media y la desviación de los 128 embeddings, para devolver 
        un vector de 256 ccas por audio o los 128 espectros en caso de que embeddings sea falso.
        
        Parametros
        ----------
        rutaAudio:str
            ruta del audio en concreto
        embeddings:boolean
            Si es true sacamos los embeddings, si false sacamos los espectros
        Return
        ------
        final_ccas: numpy.array
            array de características del audio. Embeddings o espectros.
        '''
        #1. Sacamos las ccas MFCC y espectrales
        input_batch  = vggish_input.wavfile_to_examples(rutaAudio)

        #2. producimos los embbeding cn el modelo o los espectros segun se indique
        if embeddings:
            ccas = self.model.predict(input_batch[:,:,:,None])
        else:
            # "Rompemos" los grupos de 0.96s
            ccas = input_batch.reshape(-1, input_batch.shape[-1])
        
        
        #3. Hacemos media y deviación de los embbedings
        media = np.mean(ccas,axis=0)
        desvs = np.std(ccas,axis=0)
        final_ccas = np.append(media,desvs)
        
        return final_ccas

## Letra A

In [12]:
rutaCcas = 'CaracteristicasExtraidas/Regression/embbedings/'

extractor = Ex_Ccas_Vggish(rutaCcas, dic_audios_inf)

dtst = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/A/')

Directorio de características ya existente, no se crea nuevo.
Comienzo extracción HC, quedan: 300 audios.


C:\Users\Usuario\.conda\envs\pk\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Usuario\.conda\envs\pk\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Usuario\.conda\envs\pk\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Usuario\.conda\envs\pk\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Usuario\.conda\envs\pk\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Comienzo extracción PD, quedan: 150 audios.


In [13]:
bunch_a = Bunch(data = dtst[:,:-4],
               target = dtst[:,-4], 
               UPDRS = dtst[:,-3],
               UPDRS_SPEECH = dtst[:,-2],
               HY = dtst[:,-1])


In [14]:
pickle_out = open('../CaracteristicasExtraidas/Regression/embbedings/vggish_embed_v_A_ccas.bunch',"wb")
pk.dump(bunch_a, pickle_out)
pickle_out.close()

## E

In [15]:
rutaCcas = 'CaracteristicasExtraidas/Regression/embbedings/'

extractor = Ex_Ccas_Vggish(rutaCcas, dic_audios_inf)

dtst = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/E/')

Directorio de características ya existente, no se crea nuevo.
Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [16]:
bunch_e = Bunch(data = dtst[:,:-4],
               target = dtst[:,-4], 
               UPDRS = dtst[:,-3],
               UPDRS_SPEECH = dtst[:,-2],
               HY = dtst[:,-1])
pickle_out = open('../CaracteristicasExtraidas/Regression/embbedings/vggish_embed_v_E_ccas.bunch',"wb")
pk.dump(bunch_e, pickle_out)
pickle_out.close()

## I

In [17]:
rutaCcas = 'CaracteristicasExtraidas/Regression/embbedings/'

extractor = Ex_Ccas_Vggish(rutaCcas, dic_audios_inf)

dtst = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/I/')

Directorio de características ya existente, no se crea nuevo.
Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [18]:
bunch_i = Bunch(data = dtst[:,:-4],
               target = dtst[:,-4], 
               UPDRS = dtst[:,-3],
               UPDRS_SPEECH = dtst[:,-2],
               HY = dtst[:,-1])
pickle_out = open('../CaracteristicasExtraidas/Regression/embbedings/vggish_embed_v_I_ccas.bunch',"wb")
pk.dump(bunch_i, pickle_out)
pickle_out.close()

## O

In [19]:
rutaCcas = 'CaracteristicasExtraidas/Regression/embbedings/'

extractor = Ex_Ccas_Vggish(rutaCcas, dic_audios_inf)

dtst = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/O/')

Directorio de características ya existente, no se crea nuevo.
Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [20]:
bunch_o = Bunch(data = dtst[:,:-4],
               target = dtst[:,-4], 
               UPDRS = dtst[:,-3],
               UPDRS_SPEECH = dtst[:,-2],
               HY = dtst[:,-1])
pickle_out = open('../CaracteristicasExtraidas/Regression/embbedings/vggish_embed_v_O_ccas.bunch',"wb")
pk.dump(bunch_o, pickle_out)
pickle_out.close()

## U

In [21]:
rutaCcas = 'CaracteristicasExtraidas/Regression/embbedings/'

extractor = Ex_Ccas_Vggish(rutaCcas, dic_audios_inf)

dtst = extractor.extraccion_embeddings_directorio('PC-GITA/vowels/U/')

Directorio de características ya existente, no se crea nuevo.
Comienzo extracción HC, quedan: 300 audios.
Comienzo extracción PD, quedan: 150 audios.


In [22]:
bunch_u = Bunch(data = dtst[:,:-4],
               target = dtst[:,-4], 
               UPDRS = dtst[:,-3],
               UPDRS_SPEECH = dtst[:,-2],
               HY = dtst[:,-1])
pickle_out = open('../CaracteristicasExtraidas/Regression/embbedings/vggish_embed_v_U_ccas.bunch',"wb")
pk.dump(bunch_u, pickle_out)
pickle_out.close()

---------------